In [ ]:
import os
# 🔌 Setup and Imports
import warnings

from dotenv import load_dotenv

warnings.filterwarnings("ignore")

from datetime import datetime, timezone
from hummingbot_api_client import HummingbotAPIClient
from core.notifiers import NotificationManager, NotificationMessage

load_dotenv()

print("✅ Hummingbot Status Reporter initialized")
print("🤖 Using official hummingbot-api-client")

host = os.getenv("HUMMINGBOT_API_HOST", "localhost") # Change if Hummingbot API is on a different host

print("📊 Fetching data from Hummingbot API...")

client = HummingbotAPIClient(base_url=f"http://{host}:8000")
await client.init()


In [ ]:
dbs = await client.archived_bots.list_databases()
dbs

In [ ]:
async def get_all_trades_for_db_and_controllers(db_path: str):
    all_trades = []
    all_trades_collected = False
    while not all_trades_collected:
        if len(all_trades) == 0:
            response = await client.archived_bots.get_database_trades(db_path=db_path, limit=100)
        else:
            response = await client.archived_bots.get_database_trades(db_path=db_path, limit=100, offset=response["pagination"]["offset"] + response["pagination"]["limit"])
        all_trades.extend(response["trades"])
        if not response["pagination"]["has_more"]:
            all_trades_collected = True
    controllers = await client.archived_bots.get_database_controllers(db_path=db_path)
    return all_trades, controllers

In [ ]:
all_trades, controller_configs = await get_all_trades_for_db_and_controllers(dbs[0])